# Deploying ML model to AWS ☁️

### CREATE INSTANCE:

1. Go to AWS and create EC2 ubuntu instance. Allow traffic from everywhere. Tick 3 boxes.
2. Open putty and copy instance id info (Click on instance id => Connect => SSH Client)
3. Launch instance.  ( pwd => /home/ubuntu )

4. Update and install python (1:25):
sudo apt-get update
sudo apt-get install python3-venv

Create prod folder and inside it endpoint:
mkdir prod => cd prod => mkdir endpoint

Inside endpoint is where we will have our pickle model (kmeans_model.pkl)

### Filezilla:
1. File => Site Manager
2. Connect with instance id
3. Get model and app.py and drag to folder endpoint


### Create virutal env:
python3 -m venv venv

Activate it:
source venv/bin/activate

Install al necessary libraries:
pip install numpy
pip install flask
pip install scikit-learn

python app.py

### Open Putty and open new terminal:
curl http://127.0.0.1:5000
mk
### Install GUNICORN as a service
In endpoint => pip install gunicorn
Creamos carpeta de logs (imporante): mkdir logs

pero en endpoint:
sudo nano gunicorn.conf.py

~~~
bind="0.0.0.0:8000"
workers=1
timeout=120
accesslog = "/home/ubuntu/prod/endpoint/logs/gunicorn.access.log"
errorlog = "/home/ubuntu/prod/endpoint/logs/gunicorn.error.log"
capture_output = True
loglevel = "info"
~~~

seguimos en endpoint y pegamos esto:

(2:24)
/home/ubuntu/prod/endpoint/venv/bin/gunicorn -c /home/ubuntu/prod/endpoint/gunicorn.conf.py app:app

`y hago control + c`

si queremos chequear que funciona volvemos a abrir otra terminal putty y hacemos:
curl http://127.0.0.1:8000


Seguimos en endpoint y vamos a cambiar de ruta:
cd /etc/systemd/system

Creamos el fichero:
sudo nano clase.service

~~~
[Unit]
Description=Gunicorn instance for productivization app
After=network.target
[Service]
User=ubuntu
Group=ubuntu
WorkingDirectory=/home/ubuntu/prod/endpoint
ExecStart=/home/ubuntu/prod/endpoint/venv/bin/gunicorn -c /home/ubuntu/prod/endpoint/gunicorn.conf.py app:app
Restart=always
[Install]
WantedBy=multi-user.target
~~~

sudo systemctl daemon-reload

sudo systemctl start clase.service

sudo systemctl enable clase.service

sudo systemctl status clase.service

### Install NGINX (proxy inverso)

En ruta endpoint: /etc/systemd/system:

(3:15)
sudo apt-get install nginx

Una vez en /etc/systemd/system$

cd /etc/nginx

Y entramos en sites available:

cd sites-available

sudo nano clase

~~~
server {
listen 80;
server_name 3.144.196.76; 
location / {
proxy_pass http://127.0.0.1:8000;
}
}


~~~


cd ..

cd sites-enabled

sudo ln -s ../sites-available/clase

sudo systemctl start nginx

sudo systemctl enable nginx

sudo systemctl status nginx





Hay volver a pararlo y encenderlo:

sudo systemctl stop nginx

sudo systemctl start nginx


In [ ]:
import pickle
import numpy as np
import pandas as pd

import requests
from dotenv import load_dotenv
import os

In [45]:
load_dotenv()

IP = os.environ["IP"]

data = requests.get("http://"+IP)
print(data.text)

<h2>THIS IS OUR K-MEANS HOME PAGE</h2>


### Check K-means model

In [24]:
# Read model
with open("kmeans_model.pkl", "rb") as archivo_entrada:
    pickle_kmeans = pickle.load(archivo_entrada)

pickle_kmeans

KMeans(n_clusters=2, n_init=10, random_state=42)

In [25]:
feature_names = ['Interests', 'Shower', 'WashingMachine', 'DishWasher', 'EcoMode',
       'WaterRecycling', 'GeneralRecycling', 'Meat', 'Gasoline', 'Age_Old',
       'Age_Young', 'Estimation_No', 'Estimation_Yes', 'Pet_No', 'Pet_Yes',
       'Garden_No', 'Garden_Yes']

In [61]:
type_0 = np.array([2, 1 ,0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0]).reshape(1, -1)
# type_0 = np.array([2, 1 ,0, 0, 0, 0, 0, 1, 0, False, True, True, False, False, True, True, False]).reshape(1, -1)
type_1 = np.array([0, 0, 0, 1 ,0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1]).reshape(1, -1)
# type_1 = np.array([0, 0, 0, 1 ,0, 1, 1, 1, 1, True, False, True, False, True, False, False, True]).reshape(1, -1)


# Set sample as df to prevent error.
sample_survey_df_0 = pd.DataFrame(type_0, columns=feature_names)
sample_survey_df_1 = pd.DataFrame(type_1, columns=feature_names)

# Give me a prediction of 0
print(pickle_kmeans.predict(sample_survey_df_0))
# Give me a prediction of 1
print(pickle_kmeans.predict(sample_survey_df_1))



[0]
[1]


### Create API (endpoint) for full stack

In [ ]:
load_dotenv()

IP = os.environ["IP"]

var1 = 0
var2 = 3
var1 = 0
var1 = 0
var1 = 0

data = requests.get("http://"+IP+"/api/k-means/prediction?data="+var1+"&data=0&data=1&data=0&data=0")

data.json()

In [68]:



var1 = 1000000
var2 = "pepe"


data = "http:///api/k-means/prediction?data="+str(var1)+"&data="+var2+"&data=1&data=0&data=0"

print(data)

http:///api/k-means/prediction?data=1000000&data=pepe&data=1&data=0&data=0


## Final Endpoint for Full Stack

In [ ]:

## Hay 6 parametros que hay que pasar en la url de la API:

# Interests
# WaterRecycling
# GeneralRecycling
# Meat
# Garden_No
# Garden_Yes


d = {


    # param_1
    'Interests - pregunta 4': {
        'ahorrar agua': 0,
        'ayudar al ecosistema': 0,
        'conocer mi huella': 1,
        'ahorrar dinero': 2,
    },

    # param_2
    'Reciclas agua - pregunta 7,1': {
        'si': 0,
        'no': 1,
        'buena idea voy a probarlo': 0
    },

    # param_3
    'Reciclar vidrio,cartón - pregunta 7,2': {
        'a veces': 1,
        'siempre': 0,      
        'nunca': 2
    },

    # param_4
    'Carne - - pregunta 8 ': {
        '1 vez al dia': 1,
        '1 vez al 3 dias': 1,
        '1 vez a la semana': 1,
        'Nunca': 0,
    },

# Esta es un poco distinta/rara. Si uno es True la otra tiene que ser False

        # param_5
        'Jardin NO - si el usuario pone 0 (NO tiene jardin)  ': {

            if Jardin NO = 0:
                'param_5': 1,
            if Jardin No = > 0:
                'param_5': 0,

        # param_6
        'Jardin SI - si el usuario pone >0 (TIENE jardin)  ': {

            if Jardin SI = >0:
                'param_6': 1,
            if Jardin SI = 0:
                'param_6': 0,
        
    },
}





API = "http://3.144.111.165/api/k-means/prediction?data="{param_1}"&data="{param_2}"&data="{param_3}"&data="{param_4}"&data="{param_5}"&data="{param_6}""